<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a562f3dc4ff084c35ab0c8bff7dac72efb04d5f3cf06d952bcbe198883de0e5c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 16:18:04
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 40.00 (0.0%)
Current PnL: -16.75 L (-11.27%)
CY Booked + Current PnL: -5.31 L (-3.57%)
-------------------
Total profit:  2.15 L
Total loss:  -18.91 L
-------------------
Total Booked + Current PnL: 21.67 L (17.81%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.27 L (56.89%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196380.0,31710.0,11626.0,0.14,19.26,5.92,26.31,89.0,2.73,1.44,28.78,X40N,NTT,AC
76,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.71,87023.0,-13754.0,13845.0,-1.38,-13.65,15.91,0.09,245.0,-0.99,0.64,14.33,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,212985.0,21243.0,16506.0,-0.44,11.08,7.75,19.69,99.0,1.29,1.56,18.63,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.95,112148.0,11644.0,17461.0,0.14,11.59,15.57,28.96,79.0,0.67,0.82,49.49,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.61,21.61,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-14.17,79.0,H-SC,12.51,230632.0,-40129.0,370072.0,7.73,-14.82,160.46,121.86,133.0,-0.11,1.69,80.42,AR,ATH,IT
51,PGHH,17973.08,-28.56,65.0,H-MC,4.04,206670.0,5850.0,62931.0,3.18,2.91,30.45,34.25,80.0,0.09,1.51,8.26,X40,ATH,FMCG
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85926.0,-16284.0,109676.0,2.89,-15.93,127.64,91.37,208.0,-0.15,0.63,39.49,XY25,NTT,FINANCE
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.06,159521.0,18681.0,25810.0,2.72,13.26,16.18,31.59,66.0,0.72,1.17,43.27,XY24,NTT,BANKS
59,SAIL,228.00,48.36,71.0,M-MC,7.57,234536.0,9574.0,158077.0,2.27,4.26,67.40,74.53,192.0,0.06,1.72,37.88,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RELAXO,1176.00,-35.23,45.0,H-SC,1.82,88294.0,-56866.0,132794.0,-5.20,-39.17,150.40,52.31,136.0,-0.43,0.65,18.19,X40N,NTT,FOOTWEAR
80,VAIBHAVGBL,521.00,60.88,63.0,H-SC,6.65,139500.0,-43275.0,154343.0,-2.89,-23.68,110.64,60.77,125.0,-0.28,1.02,24.94,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3791.89,59.0,M-SC,5.38,121328.0,-26040.0,77856.0,-2.66,-17.67,64.17,35.16,236.0,-0.33,0.89,23.92,XY24,NTT,AUTO
58,ROUTE,2236.84,-47.48,43.0,H-SC,17.54,86416.0,-46650.0,139510.0,-2.26,-35.06,161.44,69.78,118.0,-0.33,0.63,0.90,SR,ATH,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130889.0,7553.0,126216.0,-2.12,6.12,96.43,108.45,119.0,0.06,0.96,29.85,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.88,44.0,H-MC,2.19,136176.0,388.0,42364.0,-0.41,0.29,31.11,31.48,107.0,0.01,1.00,12.44,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.91,248165.0,434.0,74623.0,-0.22,0.18,30.07,30.30,92.0,0.01,1.82,13.49,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.42,123633.0,-18846.0,38363.0,0.62,-13.23,31.03,13.70,163.0,-0.49,0.90,45.92,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,101.92,51.0,M-SC,3.71,87023.0,-13754.0,13845.0,-1.38,-13.65,15.91,0.09,245.0,-0.99,0.64,14.33,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,1.27,226665.0,-43002.0,79333.0,-0.55,-15.95,35.00,13.47,138.0,-0.54,1.66,14.64,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.44,108728.0,-23107.0,65335.0,-0.19,-17.53,60.09,32.03,98.0,-0.35,0.80,21.78,OX40N,NTT,IT
18,CERA,9475.0,-21.00,41.0,H-SC,2.13,144072.0,-31831.0,73851.0,-1.32,-18.10,51.26,23.89,149.0,-0.43,1.05,24.43,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,71.0,M-MC,7.57,234536.0,9574.0,158077.0,2.27,4.26,67.40,74.53,192.0,0.06,1.72,37.88,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196380.0,31710.0,11626.0,0.14,19.26,5.92,26.31,89.0,2.73,1.44,28.78,X40N,NTT,AC
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,212985.0,21243.0,16506.0,-0.44,11.08,7.75,19.69,99.0,1.29,1.56,18.63,XY25,NTT,AC
17,CAMS,5211.76,1.04,68.0,H-SC,3.44,114825.0,12821.0,31106.0,0.63,12.57,27.09,43.06,122.0,0.41,0.84,31.84,X40N,ATH,MISC
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.91,248165.0,434.0,74623.0,-0.22,0.18,30.07,30.30,92.0,0.01,1.82,13.49,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.95,112148.0,11644.0,17461.0,0.14,11.59,15.57,28.96,79.0,0.67,0.82,49.49,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.65,100335.0,8836.0,69913.0,0.18,9.66,69.68,86.07,223.0,0.13,0.73,45.70,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158646.0,7701.0,101756.0,-0.41,5.10,64.14,72.51,53.0,0.08,1.16,11.43,XR,NTT,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130889.0,7553.0,126216.0,-2.12,6.12,96.43,108.45,119.0,0.06,0.96,29.85,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.34,185283.0,5475.0,106945.0,1.23,3.05,57.72,62.52,88.0,0.05,1.36,38.17,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302699.0,-13143.0,125802.0,0.19,-4.16,41.56,35.67,5.0,-0.10,2.21,9.12,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.75,218390.0,-33378.0,86592.0,0.12,-13.26,39.65,21.13,27.0,-0.39,1.60,17.47,X40,ATH,PAINTS
30,HCLTECH,1943.91,1.08,48.0,H-LC,9.75,230382.0,-11542.0,74805.0,-1.77,-4.77,32.47,26.15,8.0,-0.15,1.69,8.87,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200808.0,670.0,20221.0,-0.22,0.33,10.07,10.44,4.0,0.03,1.47,5.92,X40,NTT,FMCG
50,NESTLEIND,1377.00,-9.11,52.0,H-LC,4.18,279513.0,14087.0,42710.0,-1.21,5.31,15.28,21.40,11.0,0.33,2.04,13.03,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,69.0,H-LC,1.47,247928.0,1777.0,77428.0,-0.27,0.72,31.23,32.18,16.0,0.02,1.81,37.02,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.37,200808.0,670.0,20221.0,-0.22,0.33,10.07,10.44,4.0,0.03,1.47,5.92,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,52.0,H-LC,4.18,279513.0,14087.0,42710.0,-1.21,5.31,15.28,21.40,11.0,0.33,2.04,13.03,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.61,21.61,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.09,56.0,H-LC,5.68,215885.0,11385.0,30030.0,-0.15,5.57,13.91,20.25,10.0,0.38,1.58,14.79,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228450.0,-5694.0,54325.0,0.06,-2.43,23.78,20.77,86.0,-0.10,1.67,4.32,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.18,5.53,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200808.0,670.0,20221.0,-0.22,0.33,10.07,10.44,4.0,0.03,1.47,5.92,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,48.0,H-LC,9.75,230382.0,-11542.0,74805.0,-1.77,-4.77,32.47,26.15,8.0,-0.15,1.69,8.87,X40,ATH,IT
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302699.0,-13143.0,125802.0,0.19,-4.16,41.56,35.67,5.0,-0.10,2.21,9.12,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,65.0,H-LC,26.99,166295.0,-19800.0,232198.0,0.24,-10.64,139.63,114.13,15.0,-0.09,1.22,22.46,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200808.0,670.0,20221.0,-0.22,0.33,10.07,10.44,4.0,0.03,1.47,5.92,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,56.0,H-LC,5.68,215885.0,11385.0,30030.0,-0.15,5.57,13.91,20.25,10.0,0.38,1.58,14.79,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.75,218390.0,-33378.0,86592.0,0.12,-13.26,39.65,21.13,27.0,-0.39,1.60,17.47,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.61,21.61,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.18,5.53,X40,ATH,IT
39,INFY,2275.00,-15.65,58.0,H-LC,8.14,328063.0,15175.0,156519.0,-0.03,4.85,47.71,54.87,3.0,0.10,2.40,11.82,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.37,200808.0,670.0,20221.0,-0.22,0.33,10.07,10.44,4.0,0.03,1.47,5.92,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302699.0,-13143.0,125802.0,0.19,-4.16,41.56,35.67,5.0,-0.10,2.21,9.12,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,260971.0,-648.0,119864.0,0.02,-0.25,45.93,45.57,7.0,-0.01,1.91,11.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7038.89,66.0,L-SC,5.21,78310.0,-15500.0,92116.0,0.96,-16.52,117.63,81.67,269.0,-0.17,0.57,51.65,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.27,85094.0,-28455.0,68509.0,0.82,-25.06,80.51,35.27,268.0,-0.42,0.62,108.27,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85926.0,-16284.0,109676.0,2.89,-15.93,127.64,91.37,208.0,-0.15,0.63,39.49,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.00,49.0,H-SC,12.50,89354.0,-11609.0,106144.0,0.13,-11.50,118.79,93.63,148.0,-0.11,0.65,33.09,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,53.0,H-SC,12.99,94605.0,-3375.0,24976.0,0.40,-3.44,26.40,22.05,152.0,-0.14,0.69,37.52,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1126.48,71.0,M-SC,4.68,166715.0,-14428.0,59901.0,2.23,-7.97,35.93,25.10,235.0,-0.24,1.22,33.61,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-14.17,79.0,H-SC,12.51,230632.0,-40129.0,370072.0,7.73,-14.82,160.46,121.86,133.0,-0.11,1.69,80.42,AR,ATH,IT
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,85926.0,-16284.0,109676.0,2.89,-15.93,127.64,91.37,208.0,-0.15,0.63,39.49,XY25,NTT,FINANCE
74,TITAGARH,1548.00,2.06,70.0,H-SC,3.14,167186.0,-24829.0,103705.0,0.93,-12.93,62.03,41.08,158.0,-0.24,1.22,41.67,XY24,NTT,ENGINEERING
7,ATULAUTO,844.00,3791.89,59.0,M-SC,5.38,121328.0,-26040.0,77856.0,-2.66,-17.67,64.17,35.16,236.0,-0.33,0.89,23.92,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.04
1,25,44.03
2,50,74.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.56
LC    32.10
MC    23.36
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.34
X40      15.43
X40N     11.94
XR       11.76
AR        8.88
XY25      8.24
OX40N     8.19
X200      1.81
MH        1.71
X5K       1.44
SR        1.28
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.32
H-LC    25.38
H-MC    20.33
M-SC    13.66
M-LC     5.65
M-MC     2.72
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.75,-11.89,72.90
FMCG,11.89,-4.09,40.51
FINANCE,9.76,-15.27,60.41
MISC,7.43,-13.28,76.25
BANKS,6.37,-14.64,74.95
PAINTS,5.86,-15.47,32.69
ELECTRICAL,5.77,-4.61,62.27
AC,3.72,4.69,11.26
INSURANCE,3.43,-7.24,44.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2837342.0,22
XR,1299857.0,14
AR,1285960.0,9
X40,741391.0,10
X40N,573321.0,9
OX40N,526728.0,10
XY25,338237.0,6
SR,245654.0,2
X5K,120572.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404016.0,29
M-SC,1379600.0,17
H-LC,1223188.0,14
H-MC,1093430.0,15
M-LC,377927.0,4
M-MC,306047.0,2
L-SC,244221.0,3
L-MC,60171.0,1
L-LC,38287.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1023207.0      6
M-SC       XY24       798679.0      7
H-SC       AR         772340.0      5
           XR         756286.0      7
H-MC       XY24       493616.0      4
H-LC       X40        461053.0      5
           AR         352062.0      2
M-MC       XY24       306047.0      2
H-SC       X40N       290166.0      4
           OX40N      253427.0      4
           SR         245654.0      2
H-LC       X40N       210157.0      3
H-MC       X40        209095.0      4
M-SC       AR         161558.0      2
L-SC       XR         160625.0      2
M-LC       XY24       155599.0      2
H-MC       XY25       127980.0      2
M-SC       OX40N      124370.0      4
M-LC       X5K        120572.0      1
M-SC       XR         114074.0      2
           XY25       109676.0      1
H-MC       XR         106945.0      1
M-LC       XR         101756.0      1
L-SC       OX40N       83596.0      1
H-LC       X200        77428.0      1
H-MC       X40N        72998.0      2
M-SC       X40         71243.0      1
H-MC       OX40N       65335.0      1
H-SC       MH          62936.0      1
H-LC       XY25        62294.0      2
           XY24        60194.0      1
L-MC       XR          60171.0      1
L-LC       XY25        38287.0      1
H-MC       MH          17461.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
